<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Объявление-функций" data-toc-modified-id="Объявление-функций-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Объявление функций</a></span></li><li><span><a href="#Чтение-и-анализ-данных" data-toc-modified-id="Чтение-и-анализ-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Чтение и анализ данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li></ul></li><li><span><a href="#BERT" data-toc-modified-id="BERT-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>BERT</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Классификация комментариев по тональности

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные.  

В нашем распоряжении набор данных с разметкой о токсичности правок.

Требуется построить модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`.  

Столбец *text* содержит текст комментария,  
*toxic* — целевой признак.

## Подготовка

### Импорт библиотек
Произведем импорт всех необходимых бибилиотек:

In [1]:
# библиотеки работы с данными
import pandas as pd
import numpy as np

# откючаем предупреждения
import warnings
warnings.filterwarnings('ignore')

# разделение на выборки
from sklearn.model_selection import train_test_split  

# модели
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier

# метрики
from sklearn.metrics import f1_score

# тексты
import nltk # инструментарий естественного языка
import re # работа с регулярными выражениями
import string # содержит набор полезных констант (напр, punctuation)
from nltk.stem.wordnet import WordNetLemmatizer # лемматизация
from nltk import word_tokenize, sent_tokenize, pos_tag # токенизация
from nltk.corpus import stopwords as nltk_stopwords # стоп-слова
from sklearn.feature_extraction.text import TfidfVectorizer # счётчик величин TF-IDF

from itertools import product # аналог вложенных циклов

### Объявление функций

**Запишем все функции, которые понадобятся нам в проекте:**

Вывод общей информации о датафрейме:

In [2]:
def info(df):
    print('Строки/столбцы:', df.shape)
    print('\nПропуски:',df.isna().sum().sum())
    print('\nДубликаты:',df.duplicated().sum())
    print('\nИнформация:')
    print(df.info())
    print('\nЗначения целевого столбца toxic:')
    print(df.toxic.value_counts())

Деление датасета на 3 выборки: обучающую, валидационную и тестовую:

In [3]:
def train_test_valid_split(df, test_size, valid_size):

    train, test = train_test_split(df, test_size = test_size, shuffle = False)
    post_split_valid_size = valid_size / (1 - test_size)
    train, valid = train_test_split(train, test_size = post_split_valid_size, shuffle =False)
    
    return train, test, valid

Функция лемматизации

In [4]:
def tokenizer(text):
    
    # Определение токенов
    tokens = [ word for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    tokens = list(filter(lambda t: t not in punctuation, tokens)) # Удаление знаков препинания внутри слов
    tokens = list(filter(lambda t: t.lower() not in stop_words, tokens)) # Удаление стоп-слов
    filtered_tokens = []
    
    # Регулярные выражения
    for token in tokens: 
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    filtered_tokens = list(
        map(lambda token: wordnet_lemmatizer.lemmatize(token.lower()), filtered_tokens)) # Лемматизация токенов
    filtered_tokens = list(filter(lambda t: t not in punctuation, filtered_tokens)) # Фильтр знаков препинания
    
    # При выводе избавимся от лишних пробелов
    return ' '.join(filtered_tokens)

### Чтение и анализ данных

Прочитаем датасет из файла csv:

In [5]:
df =  pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Просмотрим общую информацию о нем:

In [6]:
info(df)

Строки/столбцы: (159571, 2)

Пропуски: 0

Дубликаты: 0

Информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None

Значения целевого столбца toxic:
0    143346
1     16225
Name: toxic, dtype: int64


Мы видим, что в датасете пропуски и дубликаты отсутствуют, все типы данных и названия столбцов корректны.  
В целевом столбце отстуствуют лишние значения, он бинарен (состоит из нулей и единиц), что соответствует логике  
Присутствует дисбаланс классов (токсичных комментариев меньше)

Посмотрим на процентное сдержание токсичных комментариев:

In [7]:
toxic_ratio = pd.Series( df['toxic'] == 1 ).sum() / df.shape[0]

print( 'Процентное содержание токсичных комментариев: {:.2%}'.format(toxic_ratio) )

Процентное содержание токсичных комментариев: 10.17%


In [8]:
# Определение стоп слов
stop_words = set(nltk_stopwords.words('english'))

# Cчётчик с указанными в нём стоп-словами
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

# Определение словаря знаков препинания 
punctuation = string.punctuation 

# Определение функций лемматизатора
wordnet_lemmatizer = WordNetLemmatizer()

In [9]:
%%time

df['text_clean'] = df['text'].map(tokenizer)
df.shape

CPU times: user 2min 44s, sys: 540 ms, total: 2min 44s
Wall time: 2min 45s


(159571, 3)

In [10]:
df.head()

,text,toxic,text_clean
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,d'aww match background colour 'm seemingly stu...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man 'm really trying edit war 's guy const...
3,"""\nMore\nI can't make any real suggestions on ...",0,ca n't make real suggestion improvement wonder...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page 's


Разделим датасет на тренировочный, валидационный и тестовый:

In [11]:
train, test, valid = train_test_valid_split(df, 0.1, 0.2)

print('Размер train:', train.shape,
     '\nРазмер test:', test.shape, 
     '\nРазмер valid:', valid.shape)

Размер train: (111698, 3) 
Размер test: (15958, 3) 
Размер valid: (31915, 3)


### Вывод
В рамках данного пункта:  
* мы провели первичный анализ данных (выяснив, что около 10% от всех комментариев являются токсичными)
* создали столбец с лемматизированным и очищенным текстом (для лемматизации и очистки текста от лишних символов и пробелов мы использовали регулярные выражения)
* и подготовили выборки (train, test, valid) для обучения моделей.

## Обучение

Чтобы алгоритмы умели определять тематику и тональность текста, их нужно обучить на корпусе (с разметкой эмоций и ключевых слов).  
Создадим корпус постов: для этого преобразуем столбец 'text_clean' в список текстов и переведем в кодировку Unicode (U)

In [12]:
train_corpus = train['text_clean'].values.astype('U')

Чтобы посчитать TF-IDF для корпуса текстов, вызовем функцию fit_transform()  
fit() вызовем только на тренировочной выборке, т.к. данные уже поделены

In [13]:
tfidf_train = count_tf_idf.fit_transform(train_corpus)

### Логистическая регрессия

Без подбора параметров (по дефолту penalty = l2):

In [14]:
model = LogisticRegression()
model.fit(tfidf_train, train['toxic'])

valid_corpus = valid['text_clean'].values.astype('U')
tfidf_valid = count_tf_idf.transform(valid_corpus)
 
pred_valid = model.predict(tfidf_valid)

f1_lr = f1_score(valid['toxic'],  pred_valid)

print('Метрика F1 на ', f1_lr)

Метрика F1 на  0.7266475644699141


По заданию требуется достичь значения метрики выше 0.75, а у нас получилось 0.73.  
Это значение близко к требуемому, но решение не подходит.  
Попробуем подобрать параметры логистической регрессии и получить новый результат метрики F1

In [16]:
%%time

buf = dict()

for i in product([{'penalty':'l1'},{'penalty':'l2'}],
              [{'max_iter':2},{ 'max_iter':5}, {'max_iter':100}]
              ):
    rfr = LogisticRegression(random_state = 123, **i[0],**i[1])
    rfr.fit(tfidf_train, train['toxic'])
    pred_2 = rfr.predict(tfidf_valid)
    buf[str(i)] = f1_score(valid['toxic'], pred_2)
    
buf

CPU times: user 10.7 s, sys: 4.76 s, total: 15.4 s
Wall time: 15.4 s


{"({'penalty': 'l1'}, {'max_iter': 2})": 0.7492977528089888,
 "({'penalty': 'l1'}, {'max_iter': 5})": 0.7639639639639639,
 "({'penalty': 'l1'}, {'max_iter': 100})": 0.7647583377920456,
 "({'penalty': 'l2'}, {'max_iter': 2})": 0.0,
 "({'penalty': 'l2'}, {'max_iter': 5})": 0.58335144533797,
 "({'penalty': 'l2'}, {'max_iter': 100})": 0.7266475644699141}

Лучшее значение метрики F1 = 0.76 достигнуто при макс итерации = 5 и штрафе penalty = l1.  
Оно подходит по заданию, т.к. больше чем 0.75

### Дерево решений

In [17]:
%%time
 
buf_1 = dict()

for i in product([{'criterion':'gini'}, {'criterion':'entropy'}],
              [{'max_depth':5}, { 'max_depth':10}, {'max_depth':100}]
              ):
    dtc = DecisionTreeClassifier(random_state = 123, **i[0],**i[1])
    dtc.fit(tfidf_train, train['toxic'])
    pred_3 = dtc.predict(tfidf_valid)
    buf_1[str(i)] = f1_score(valid['toxic'],pred_3)
    
buf_1

CPU times: user 3min 26s, sys: 55.7 ms, total: 3min 26s
Wall time: 3min 27s


{"({'criterion': 'gini'}, {'max_depth': 5})": 0.5118805159538357,
 "({'criterion': 'gini'}, {'max_depth': 10})": 0.5924679150010519,
 "({'criterion': 'gini'}, {'max_depth': 100})": 0.7242570836212854,
 "({'criterion': 'entropy'}, {'max_depth': 5})": 0.5124378109452736,
 "({'criterion': 'entropy'}, {'max_depth': 10})": 0.586722866174921,
 "({'criterion': 'entropy'}, {'max_depth': 100})": 0.7010515773660491}

Лучший показатель F1 при переборе параметров модели дерева решений, оказался ниже чем требуется (0.72)

### Тестирование лучшей модели

Проверим на тестовой выборке лучшую модель Логистической регрессии, полученную на валидационной выборке:

In [23]:
test_corpus = test['text_clean'].values.astype('U')
tfidf_test = count_tf_idf.transform(test_corpus) 

best_model = LogisticRegression(random_state = 123 , max_iter = 5, penalty = 'l1')
best_model.fit(tfidf_train, train['toxic'])
pred_test = best_model.predict(tfidf_test)

f1 = f1_score(test['toxic'], pred_test)
f1

0.7605633802816901

Успех. При заданных гиперпараметрах модели логистической регрессии,  
на тесте мы получили значение метрики F1 = 0.76, что выше требуемого порога в 0.75

## BERT

## Выводы

В рамках данного проекта мы помогли магазину "Викишоп" создать инструмент, который будет помогать классифицировать комментарии на позитивные и негативные с целью отправки негативных на дальнейшую модерацию. В поисках лучшего результата, были перебраны гиперпараметры двух моделей: дерево решений и логистическая решрессия. Больше всего нам подошла модель Логистическая регрессия, гиперпараметры которой мы подобрали перебором (max_iter = 5, penalty = 'l1'). В результате метрика качества предсказаний оказалась лучше требуемого порога: F1 = 0.76 против требуемых <= 0.75, поэтому модель принимаем как удачную.